In [36]:
import h5py
import numpy as np

from keras.layers import Input, Dense, Activation, MaxPooling2D, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Dropout
from keras.models import Model

from keras.utils import plot_model
from keras.metrics import MeanSquaredError, CategoricalAccuracy

from keras.initializers import glorot_uniform

import keras.backend as K
K.set_image_data_format('channels_last')

In [14]:
def get_data(filepath):
    data_file = h5py.File(filepath,'r')

    train_data = data_file['train_data'][:]
    train_labels = data_file['train_labels'][:]

    test_data = data_file['test_data'][:]
    test_labels = data_file['test_labels'][:]

    return train_data, train_labels, test_data, test_labels

In [23]:
def iShameModel(input_shape, conv_layers, conn_layers, output_classes=None):

    # input layer
    X_input = Input(input_shape)
    X = X_input

    # convlutional layers
    # CONV2D -> RELU -> MAXPOOL
    # number of filters, filter_size, and strides retrieved from input conv array

    for idx, layer in enumerate(conv_layers):        
        X = Conv2D(layer["num_filters"], layer["filter_size"], strides = layer["filter_size"], \
                    name = 'conv_'+str(idx), kernel_initializer = glorot_uniform(seed=0))(X)
        #X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
        X = Activation(layer["activation"])(X)
        X = MaxPooling2D(layer["filter_size"], strides=layer["stride_size"], name="max_pool_"+str(idx))(X)

    # add fully connected layers
    X = Flatten()(X)

    for idx, layer in enumerate(conn_layers):
        X = Dense(layer["hidden_units"], activation=layer["activation"], name="fully_connected_"+str(idx))(X)

    # if classification, add softmax output layer
    # otherwise, add singular output layer with one unit with NO activation
    if output_classes is not None:
        X = Dense(output_classes, activation='softmax', name='class_output_'+str(output_classes), kernel_initializer = glorot_uniform(seed=0))(X)
    else:
        X = Dense(1, name = 'regression_output')(X)

    model = Model(inputs=X_input, outputs=X, name='iShame')

    return model

In [37]:
# get training, test data

data_save_path = '../img/label_data'
data_set_name='coffee_label_data.h5'
filepath=os.path.join(data_save_path,data_set_name)
train_data, train_labels, test_data, test_labels  = get_data(filepath)

In [38]:
print(test_data.shape)

(97, 540, 960, 3)


In [18]:
conv_layers = [{"num_filters": 16,
                "filter_size": (3,3),
                "stride_size": (1,1),
                "activation": "relu",
                "pool_size": (2,2)
                },
                {"num_filters": 32,
                "filter_size": (3,3),
                "stride_size": (1,1),
                "activation": "relu",
                "pool_size": (2,2)
                },
                {"num_filters": 64,
                "filter_size": (3,3),
                "stride_size": (1,1),
                "activation": "relu",
                "pool_size": (2,2)
                }]
conn_layers = [{"hidden_units": 500,
                    "activation": "relu"},
                {"hidden_units": 100,
                    "activation": "relu"},
                {"hidden_units": 20,
                    "activation": "relu"}
                ]

In [25]:
regression_output = False
if(regression_output):
    num_classes=None
    loss="mean_squared_error"
    metrics=[MeanSquaredError()]
else:
    num_classes=train_labels.shape[-1]
    loss="categorical_crossentropy"
    metrics=[CategoricalAccuracy()]
batch_size=32
epochs=10

In [26]:
model = iShameModel(input_shape=train_data.shape[1:],conv_layers=conv_layers,conn_layers=conn_layers,output_classes=num_classes)


In [27]:
model.compile(optimizer='adam', loss=loss, metrics=metrics)

In [28]:
model.fit(train_data, train_labels, epochs = epochs, batch_size = batch_size)

Epoch 1/10
387/387 [==============================] - 80s 206ms/step - loss: 0.1018 - categorical_accuracy: 0.9173
Epoch 2/10
387/387 [==============================] - 61s 157ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 3/10
387/387 [==============================] - 61s 158ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 4/10
387/387 [==============================] - 62s 160ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 5/10
387/387 [==============================] - 63s 162ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 6/10
387/387 [==============================] - 61s 159ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 7/10
387/387 [==============================] - 61s 158ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 8/10
387/387 [==============================] - 62s 160ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 9/10
387/387 [==============================] 

In [43]:
results = model.evaluate(test_data, test_labels)
print("Loss:",results[0])
print("Result:",results[1])

97/97 [==============================] - 10s 101ms/step
Loss: 0.0
Result: 1.0


In [40]:
model.summary()

Model: "iShame"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 540, 960, 3)       0         
_________________________________________________________________
conv_0 (Conv2D)              (None, 180, 320, 16)      448       
_________________________________________________________________
activation_7 (Activation)    (None, 180, 320, 16)      0         
_________________________________________________________________
max_pool_0 (MaxPooling2D)    (None, 178, 318, 16)      0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 59, 106, 32)       4640      
_________________________________________________________________
activation_8 (Activation)    (None, 59, 106, 32)       0         
_________________________________________________________________
max_pool_1 (MaxPooling2D)    (None, 57, 104, 32)       0    

In [42]:
plot_model(model)

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.